# Cloud Service Scoring Evaluation

This notebook provides a comprehensive evaluation framework for analyzing Cloud Elevation Index (CEI) scores generated by different AI models. It includes:

- **Model Comparison**: Compare scores between Gemini 1.5 Pro and Gemini 2.0 Flash
- **Ground Truth Validation**: Validate against known service categories
- **Anomaly Detection**: Identify statistical outliers
- **Visualizations**: Interactive plots and charts
- **Comprehensive Reporting**: Generate detailed evaluation reports

## 1. Import Required Libraries

First, let's import all the necessary libraries for data processing, analysis, and visualization.

In [ ]:
import json
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Import our custom utils functions
from utils import (
    load_old_format_scores,
    load_new_format_scores,
    load_multiple_score_files,
    compare_model_scores_unified,
    validate_dataframe_against_ground_truth,
    export_comparison_results_to_file
)

## 2. Load Ground Truth Data

Load ground truth services with expected score ranges from the configuration file.

**File Structure:**
- Configuration files: `config/` directory (prompts, ground truth, input data)
- Score results: `scores/` directory (all scoring outputs and logs)
- Utilities: `utils/` directory (reusable analysis functions)

In [ ]:
# Load ground truth data from JSON file in config directory
with open('../config/ground_truth.json', 'r') as f:
    config = json.load(f)
    ground_truth = config['ground_truth']

# Convert list ranges back to tuples for compatibility
for service_name, service_data in ground_truth.items():
    service_data['expected_range'] = tuple(service_data['expected_range'])

print(f"Loaded ground truth data for {len(ground_truth)} services")
print("Categories:", set(data['category'] for data in ground_truth.values()))

## 3. Score Data Loading and Comparison

The utility functions for loading different NDJSON formats and performing cross-model score comparisons are now imported from the utils package.

In [ ]:
# Example: Load different format files and compare scores with run identifiers
import os

# Check available data files in both config and scores directories
config_files = [f"../config/{f}" for f in os.listdir('../config') if f.endswith('.ndjson')]
scores_files = [f"../results/scores/{f}" for f in os.listdir('../results/scores') if f.endswith('.ndjson')]
available_files = config_files + scores_files

print("Available NDJSON files:")
print("  Config files:", [f for f in config_files])
print("  Score files:", [f for f in scores_files])

# Import the new helper function
from utils.data_loaders import load_multiple_score_files

# ===== CUSTOMIZE THIS SECTION =====
# Define your file configurations with run identifiers
# Format: (file_path, format_type, run_identifier)
file_configs = [
    # Old format files
    ('../results/scores/cei_scores.ndjson', 'old', 'baseline'),
    
    # New format files - you can now distinguish different runs of the same model
    ('../results/scores/2025-10-18-19-50-26-llama-3-3-70b.ndjson', 'new', 'run1'),
    ('../results/scores/2025-10-18-21-10-00-claude-sonnet-4-5.ndjson', 'new', 'run1'),
    ('../results/scores/2025-10-23-15-23-13-llama-3-3-70b.ndjson', 'new', 'run2'),
    
    # Example of how to compare same model with different configurations:
    # ('../results/scores/llama-with-tools.ndjson', 'new', 'with_tools'),
    # ('../results/scores/llama-without-tools.ndjson', 'new', 'without_tools'),
]

# Load all configured files
dataframes_to_compare = load_multiple_score_files(file_configs)

print(f"\nLoaded {len(dataframes_to_compare)} datasets for comparison")

# Alternative: Manual loading with specific identifiers (if you prefer more control)
"""
dataframes_to_compare = []

# Load with specific run identifiers
if os.path.exists('../results/scores/2025-10-18-19-50-26-llama-3-3-70b.ndjson'):
    df1 = load_new_format_scores('../results/scores/2025-10-18-19-50-26-llama-3-3-70b.ndjson', 'with_tools')
    dataframes_to_compare.append(df1)

if os.path.exists('../results/scores/2025-10-23-15-23-13-llama-3-3-70b.ndjson'):
    df2 = load_new_format_scores('../results/scores/2025-10-23-15-23-13-llama-3-3-70b.ndjson', 'without_tools')
    dataframes_to_compare.append(df2)
"""

In [ ]:
# Example: Validate individual dataframes against ground truth
print("\n=== GROUND TRUTH VALIDATION ===")

# Validate each loaded dataframe against ground truth
for i, df in enumerate(dataframes_to_compare):
    print(f"\nValidating Dataset {i+1}:")
    
    # Run ground truth validation
    validation_results = validate_dataframe_against_ground_truth(df, ground_truth)
    
    if 'error' in validation_results:
        print(f"Error: {validation_results['error']}")
        continue
    
    # Display summary
    summary = validation_results['validation_summary']
    print(f"  Score columns analyzed: {', '.join(validation_results['score_columns_analyzed'])}")
    print(f"  Services found in data: {summary['services_found_in_data']}/{summary['total_services_in_ground_truth']}")
    print(f"  Services within expected range: {summary['services_within_expected_range']}")
    print(f"  Services outside expected range: {summary['services_outside_expected_range']}")
    print(f"  Accuracy rate: {summary['accuracy_rate']:.2%}")
    
    # Show services that don't meet ground truth (outside expected range)
    if validation_results['services_outside_range']:
        print(f"\n  === SERVICES OUTSIDE EXPECTED RANGE ===")
        for j, service in enumerate(validation_results['services_outside_range'][:5], 1):  # Show top 5
            print(f"  {j}. {service['service_name']} ({service['provider']})")
            print(f"     Expected ({service['expected_category']}): {service['expected_range']}")
            for score_col, analysis in service['scores_analysis'].items():
                print(f"     {score_col}: {analysis['actual_score']:.2f} (deviation: {analysis['deviation_from_range']:.2f})")
            print()
    
    # Show services not found in data
    if validation_results['services_not_found']:
        print(f"  === SERVICES NOT FOUND IN DATA ===")
        for service in validation_results['services_not_found'][:3]:  # Show first 3
            print(f"  - {service['service_name']} ({service['provider']}) - Expected: {service['expected_category']}")
    
    print("-" * 60)

## 4. Usage: Loading and Comparing Multiple Score Files

Examples of how to use the imported utility functions with different data formats.

In [ ]:
# Available score columns in your data:
print("=== AVAILABLE SCORE COLUMNS ===")
all_available_columns = set()
for i, df in enumerate(dataframes_to_compare):
    score_cols = [col for col in df.columns if 'score' in col.lower()]
    print(f"Dataset {i+1}: {score_cols}")
    all_available_columns.update(score_cols)

print(f"\nAll unique score columns: {sorted(list(all_available_columns))}")

## How to Compare Different Runs of the Same Model

With the updated code, you can now compare different runs of the same model by using run identifiers. Here are some common scenarios:

### Scenario 1: Same model, different configurations
```python
file_configs = [
    ('../results/scores/llama-with-tools.ndjson', 'new', 'with_tools'),
    ('../results/scores/llama-without-tools.ndjson', 'new', 'without_tools'),
]
columns_to_compare = ['llama_3_3_70b_with_tools_score', 'llama_3_3_70b_without_tools_score']
```

### Scenario 2: Same model, different time periods
```python
file_configs = [
    ('../results/scores/2025-10-18-llama.ndjson', 'new', 'oct18'),
    ('../results/scores/2025-10-23-llama.ndjson', 'new', 'oct23'),
]
columns_to_compare = ['llama_3_3_70b_oct18_score', 'llama_3_3_70b_oct23_score']
```

### Scenario 3: Different versions of prompts
```python
file_configs = [
    ('../results/scores/llama-prompt-v1.ndjson', 'new', 'prompt_v1'),
    ('../results/scores/llama-prompt-v2.ndjson', 'new', 'prompt_v2'),
]
columns_to_compare = ['llama_3_3_70b_prompt_v1_score', 'llama_3_3_70b_prompt_v2_score']
```

In [ ]:
# ===== EXAMPLE: Comparing Same Model with Different Configurations =====
# This demonstrates how to compare Llama 3.3 70B with and without tools

# If you have two different runs of the same model, configure them like this:
example_configs = [
    # First run - let's say this was with tools enabled
    ('../results/scores/2025-10-18-19-50-26-llama-3-3-70b.ndjson', 'new', 'with_tools'),
    
    # Second run - let's say this was without tools  
    ('../results/scores/2025-10-23-15-23-13-llama-3-3-70b.ndjson', 'new', 'without_tools'),
]

# Then you can compare specifically these two configurations:
example_columns = ['llama_3_3_70b_with_tools_score', 'llama_3_3_70b_without_tools_score']

print("Example configuration for comparing same model with different setups:")
print("File configs:", example_configs)
print("Columns to compare:", example_columns)
print("\nThis allows you to see which services show different scores when using tools vs not using tools!")

# The key insight: Each run gets its own unique column name, so no data is lost!

In [ ]:
# CUSTOMIZE THIS: Specify which columns you want to compare
# Now you can compare different runs of the same model!

# Example options:
# columns_to_compare = ['claude_simple_score', 'gemini_1_5_pro_score']
# columns_to_compare = ['cei_score', 'claude_simple_score'] 
# columns_to_compare = None  # Compare all available columns

# Compare different runs of the same model:
# columns_to_compare = ['llama_3_3_70b_with_tools_score', 'llama_3_3_70b_without_tools_score']
# columns_to_compare = ['llama_3_3_70b_run1_score', 'llama_3_3_70b_run2_score']

# Based on the current file configuration above, these columns should be available:
columns_to_compare = ['llama_3_3_70b_run1_score', 'llama_3_3_70b_run2_score']  # Modify this list as needed

print(f"Configured to compare: {columns_to_compare}")

In [ ]:
# Run comparison analysis if we have data to compare
if len(dataframes_to_compare) >= 2:
    print(f"\n=== RUNNING COMPARISON ANALYSIS ===")
    print(f"Comparing {len(dataframes_to_compare)} datasets")

    if 'columns_to_compare' not in locals() or columns_to_compare is None:
        columns_to_compare = None  # Will use all available score columns
        print("Using all available score columns for comparison")
    else:
        print(f"Using specified columns for comparison: {columns_to_compare}")
    
    # Run comparison with configurable thresholds
    comparison_results = compare_model_scores_unified(
        dataframes=dataframes_to_compare,
        disagreement_threshold=2.0,  # Configurable threshold
        show_similar=True,           # Set to True to see similar services too
        similarity_threshold=1,    # Configurable similarity threshold
        columns_to_compare=columns_to_compare  # Specify which columns to compare
    )
    
    # Check for errors
    if 'error' in comparison_results:
        print(f"❌ Error: {comparison_results['error']}")
    else:
        # Display results
        print(f"\n=== ANALYSIS SUMMARY ===")
        summary = comparison_results['analysis_summary']
        print(f"Total services compared: {summary['total_services_compared']}")
        print(f"Score columns analyzed: {', '.join(comparison_results['score_columns_analyzed'])}")
        print(f"Services with high disagreement: {summary['services_with_high_disagreement']}")
        print(f"Services with high similarity: {summary['services_with_high_similarity']}")
        
        # Show high disagreement services
        print(f"\n=== HIGH DISAGREEMENT SERVICES (deviation > {comparison_results['disagreement_threshold']}) ===")
        for i, service in enumerate(comparison_results['high_disagreement_services'][:10], 1):  # Show top 10
            print(f"{i}. {service['service_name']} ({service['provider']})")
            print(f"   Median Score: {service['median_score']:.2f}")
            print(f"   Max Deviation: {service['max_deviation']:.2f}")
            print(f"   Scores: {', '.join([f'{k}={v:.2f}' for k, v in service['scores'].items()])}")
            print(f"   Deviations: {', '.join([f'{k}={v:.2f}' for k, v in service['deviations'].items()])}")
            print()
        
        # Show high similarity services (if requested)
        if comparison_results['high_similarity_services']:
            print(f"\n=== HIGH SIMILARITY SERVICES (deviation <= {comparison_results['similarity_threshold']}) ===")
            for i, service in enumerate(comparison_results['high_similarity_services'][:5], 1):  # Show top 5
                print(f"{i}. {service['service_name']} ({service['provider']})")
                print(f"   Median Score: {service['median_score']:.2f}")
                print(f"   Max Deviation: {service['max_deviation']:.2f}")
                print(f"   Scores: {', '.join([f'{k}={v:.2f}' for k, v in service['scores'].items()])}")
                print()
        
        # Export all results to text file
        print(f"\n=== EXPORTING RESULTS ===")
        exported_file = export_comparison_results_to_file(comparison_results)
        if exported_file:
            print(f"Complete analysis exported to: {exported_file}")
            print("This file contains ALL services that meet the criteria, not just the top ones shown above.")

else:
    print("Need at least 2 datasets to run comparison analysis")
    print("Please ensure you have the appropriate NDJSON files in the current directory")

In [ ]:
# Optional: Visualize disagreement analysis
def visualize_disagreement_analysis(comparison_results):
    """Create visualizations for the disagreement analysis."""
    if 'error' in comparison_results:
        print("Cannot create visualizations:", comparison_results['error'])
        return
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # 1. Disagreement distribution
    disagreement_services = comparison_results['high_disagreement_services']
    if disagreement_services:
        max_deviations = [s['max_deviation'] for s in disagreement_services]
        axes[0].hist(max_deviations, bins=20, edgecolor='black', alpha=0.7)
        axes[0].axvline(comparison_results['disagreement_threshold'], 
                       color='red', linestyle='--', 
                       label=f'Threshold: {comparison_results["disagreement_threshold"]}')
        axes[0].set_xlabel('Max Deviation from Median')
        axes[0].set_ylabel('Number of Services')
        axes[0].set_title('Distribution of High Disagreement Services')
        axes[0].legend()
    
    # 2. Top disagreement services
    if len(disagreement_services) >= 10:
        top_services = disagreement_services[:10]
        service_names = [s['service_name'][:15] + '...' if len(s['service_name']) > 15 
                        else s['service_name'] for s in top_services]
        deviations = [s['max_deviation'] for s in top_services]
        
        axes[1].barh(range(len(service_names)), deviations)
        axes[1].set_yticks(range(len(service_names)))
        axes[1].set_yticklabels(service_names)
        axes[1].set_xlabel('Max Deviation from Median')
        axes[1].set_title('Top 10 Services with Highest Disagreement')
        axes[1].invert_yaxis()
    
    plt.tight_layout()
    plt.show()

# Create visualization if we have comparison results
if 'comparison_results' in locals():
    visualize_disagreement_analysis(comparison_results)